In [57]:
import json
import os
import math
import librosa
import boto3
import soundfile as sf
from pydub import AudioSegment
from app import *
from flask_mysqldb import MySQL
from io import BytesIO
import io
from werkzeug.utils import secure_filename
from six.moves.urllib.request import urlopen

In [58]:
with app.app_context():
     cur = mysql.connection.cursor()
     app.config['MYSQL_HOST'] = '192.168.0.100'
     app.config['MYSQL_USER'] = 'idp'
     app.config['MYSQL_PASSWORD'] = 'idp2g7'
     app.config['MYSQL_DB'] = 'idp2g7'
     query = 'SELECT s3_object_url FROM test WHERE normal IS NULL'
     cur.execute(query)
     records = cur.fetchall()
     print(records)

(('https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000000.wav',),)


In [88]:
BUCKET_NAME = "idp2g7"
ACCESS_KEY_ID = ""
SECRET_ACCESS_KEY = ""
s3 = boto3.resource('s3',aws_access_key_id= ACCESS_KEY_ID,aws_secret_access_key= SECRET_ACCESS_KEY)
bucket = s3.Bucket(BUCKET_NAME)
location = "ap-southeast-1"
for row in records:
    
    for obj in bucket.objects.filter(Prefix="test/"):
        obj_key = obj.key
        obj_url = "https://%s.s3.%s.amazonaws.com/%s" % ('idp2g7', location, obj_key)
        
        if obj_url==row[0]:
            print(obj_key)
            s3_client = boto3.client('s3',aws_access_key_id= ACCESS_KEY_ID,aws_secret_access_key= SECRET_ACCESS_KEY)
            object = s3_client.get_object(Bucket=BUCKET_NAME, Key=obj_key)
            data = io.BytesIO(object['Body'].read())
            audio = AudioSegment.from_wav(data)
            if audio.duration_seconds>5.0:
               first_5_seconds = audio[:5000]
               first_5_seconds.export(data, format="wav")
               print(first_5_seconds)
               s3_client.put_object(Body=data, Bucket= BUCKET_NAME, Key=obj_key)

test/00000000.wav


In [81]:
JSON_PATH = "data_nolabel.json" #change ur json file name here

SAMPLE_RATE = 22050
TRACK_DURATION = 5  # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION #=220500

In [90]:
def save_mfcc(json_path, num_mfcc=13, n_fft=2048, hop_length=512):
    """Extracts MFCCs from audio and saves them into a json file
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mfcc": [],
        "obj_url": []
    }

    #samples_per_segment = int(SAMPLES_PER_TRACK / num_segments) #=44100
    num_mfcc_vectors_per_track = math.ceil(SAMPLES_PER_TRACK/ hop_length) #=87

    # loop through all genre sub-folder
    for row in records:
        for obj in bucket.objects.filter(Prefix="test/"):
            obj_key = obj.key
            obj_url = "https://%s.s3.%s.amazonaws.com/%s" % ('idp2g7', location, obj_key)
    
        
            if row[0]==obj_url:
                url = "https://s3-%s.amazonaws.com/%s/%s" % (location, 'idp2g7', obj_key)
                print(url)
                signal, sample_rate = sf.read(io.BytesIO(urlopen(url).read()),dtype='float32')
                signal = signal.T
                signal = librosa.resample(signal, orig_sr=16000, target_sr=SAMPLE_RATE)
                signal = librosa.to_mono(signal)
                sample_rate = SAMPLE_RATE

                # calculate start and finish sample for current segment
                start = 0
                finish = SAMPLES_PER_TRACK

                # extract mfcc
                mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                                hop_length=hop_length)
                mfcc = mfcc.T

                # store only mfcc feature with expected number of vectors
                if len(mfcc) == num_mfcc_vectors_per_track:
                    data["mfcc"].append(mfcc.tolist())
                    data["obj_url"].append(obj_url)
                    print("{}".format(row[0]))
                    print(mfcc.shape)

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)


if __name__ == "__main__":
    save_mfcc(JSON_PATH)
    



https://s3-ap-southeast-1.amazonaws.com/idp2g7/test/00000000.wav
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000000.wav
(216, 13)


<ipython-input-90-828bc0366402>:41: FutureWarning: Pass y=[ 0.03937753 -0.00297339 -0.02199767 ...  0.00576363  0.00666459
  0.00430795], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
